In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
print(df.shape)


In [ ]:
df.describe()

In [ ]:
df = df[df.weight > 0]
df = df.dropna()
df = df.sample(n=10000, random_state=1).reset_index()
df['profit'] = df['weight']*df['resp']
df['action'] = df['profit'].apply(lambda x: 1 if x >= 0 else 0)

In [ ]:
features = [x for x in df.columns if x.startswith("feature")  ]


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)

X = df[features]
pca.fit(X)

print(pca.explained_variance_ratio_)
import matplotlib.pyplot as plt
import seaborn as sns
#plt.figure(figsize=(4,4))
sns.lineplot(data=pca.explained_variance_ratio_)

In [ ]:
X_reduced = PCA(n_components=2).fit_transform(X)
print(X_reduced.shape)

In [ ]:
plt.figure(figsize=(8,8))
sns.scatterplot(X_reduced[:, 0], X_reduced[:, 1],hue=df['action'])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
X_reduced = PCA(n_components=3).fit_transform(X)
fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=df['action'])


In [ ]:
X = df[features]
X_reduced = PCA(n_components=2).fit_transform(X)

a = X_reduced[:, 0]
b = X_reduced[:, 1]
i = a*a
j = a*b
k = b*b

print(i.shape,j.shape,k.shape)
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(i,j,k, c=df['action'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

clf = RandomForestClassifier(max_depth=5)
data = list(zip(i,j,k))
label = df['action']
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)
clf.fit(X_train,y_train )
predictions = clf.predict(X_test)
errors = abs(predictions - y_test)
print('Mean Absolute Error:', round(np.mean(errors), 2))
# Calculate mean absolute percentage error (MAPE)
probs = clf.predict_proba(X_test)[:, 1]
from sklearn.metrics import roc_auc_score,plot_roc_curve

# Calculate roc auc
roc_value = roc_auc_score(y_test, probs)
print("roc value: " , roc_value)
plot_roc_curve(clf, X_test, y_test)